## ComCam "Warm" pixels

Investigating ComCam pixels that seems to have amplification.
Craig Lage - 30-Aug-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils

In [ ]:
butler = Butler('/repo/embargo_old', collections=["LSSTComCam/raw/all", "LSSTComCam/calib",
                                              "u/abrought/LSSTComCam/calibs/w_2024_28/bias.08132024a",
                                            "u/abrought/LSSTComCam/calibs/w_2024_28/defects.08132024a",
                                            "u/abrought/LSSTComCam/calibs/w_2024_28/dark.08132024c",
                                            "u/abrought/LSSTComCam/calibs/w_2024_28/flat_r03.08132024a",
                                             "u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a",
                                                ])

# Define a plotting subroutine

In [ ]:
import astropy.visualization as vis
import matplotlib.colors as colors
import matplotlib
from lsst.summit.utils import getQuantiles
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage


def localPlot(
    imageData: np.ndarray,
    stretch: str = "linear",
    percentile: float = 99.0,
    cmap: str = "gray",
    center: tuple = (1000,1000),
    extent: tuple = (50,50)
) -> matplotlib.figure.Figure:

    # Copied and modified from lsst.summit.utils.plotting.plot.py
    xmin = int(center[0] - extent[0] / 2)
    xmax = int(center[0] + extent[0] / 2)
    ymin = int(center[1] - extent[1] / 2)
    ymax = int(center[1] + extent[1] / 2)
    figure, axs = plt.subplots(2, 2, figsize=(10, 10))
    

    stretchData = imageData[ymin:ymax, xmin:xmax]
    interval = vis.PercentileInterval(percentile)
    match stretch:
        case "ccs":
            quantiles = getQuantiles(stretchData, 256)
            norm = colors.BoundaryNorm(quantiles, 256)
        case "asinh":
            norm = vis.ImageNormalize(stretchData, interval=interval, stretch=vis.AsinhStretch(a=0.1))
        case "power":
            norm = vis.ImageNormalize(stretchData, interval=interval, stretch=vis.PowerStretch(a=2))
        case "log":
            norm = vis.ImageNormalize(stretchData, interval=interval, stretch=vis.LogStretch(a=1))
        case "linear":
            norm = vis.ImageNormalize(stretchData, interval=interval, stretch=vis.LinearStretch())
        case "sqrt":
            norm = vis.ImageNormalize(stretchData, interval=interval, stretch=vis.SqrtStretch())
        case _:
            raise ValueError(
                f"Invalid value for stretch : {stretch}. "
                "Accepted options are: ccs, asinh, power, log, linear, sqrt."
            )

    im = axs[0][0].imshow(imageData, cmap=cmap, origin="lower", norm=norm, aspect="equal")
    div = make_axes_locatable(axs[0][0])
    cax = div.append_axes("right", size="5%", pad=0.05)
    figure.colorbar(im, cax=cax)
    axs[0][0].set_xlim(xmin, xmax)
    axs[0][0].set_ylim(ymin, ymax)

    less_stamp = np.zeros_like(stretchData)
    more_stamp = np.zeros_like(stretchData)
    med = np.median(stretchData)
    sig = np.sqrt(med)
    
    for i in range(stretchData.shape[0]):
        for j in range(stretchData.shape[1]):
            if stretchData[i,j] < (med - 3.0 * sig):
                less_stamp[i,j] = (med - stretchData[i,j])
            if stretchData[i,j] > (med + 3.0 * sig):
                more_stamp[i,j] = (stretchData[i,j] - med)
    
    axs[0][1].plot(stretchData[int(stretchData.shape[1] / 2),:], marker='x')
    axs[0][1].axhline(med, ls='--', color='red')
    axs[0][1].yaxis.set_label_position("right")
    axs[0][1].yaxis.tick_right()
    axs[0][1].set_ylabel("Signal(e-)")
    
    axs[1][0].imshow(less_stamp, origin="lower",)
    axs[1][0].set_title("Pixels < median-3*sig")
    axs[1][0].text(5, 25, f"Sum = {np.sum(less_stamp):.0f}", color='white', fontsize=12)
    
    axs[1][1].imshow(more_stamp, origin="lower",)
    axs[1][1].set_title("Pixels > median+3*sig")
    axs[1][1].text(5, 25, f"Sum = {np.sum(more_stamp):.0f}", color='white', fontsize=12)
    
    return figure

## Define a simple ISR
### Just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=True
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result

In [ ]:
expId = 2024072900237
#expId = 2024072900244
#expId = 2024072900232
#expId = 2024072900075 # Dark
instrument = "LSSTComCam"
detector = 2
mData = butler.get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
biasExp = butler.get('bias', detector=detector, exposure=expId, instrument=instrument) # This is a bias image associated with the data
ptc = butler.get('ptc', detector=detector, instrument=instrument,
                 collections="u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a")
isrResult = isrTask.run(exp, bias=biasExp, ptc=ptc) # This runs the ISR

In [ ]:
center=(3159, 558)

fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
fig.suptitle(f"Warm pixels, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center}", fontsize=18)

In [ ]:
expIdF = 2024072900237 # Flat
expIdD = 2024072900075 # Dark
instrument = "LSSTComCam"
detector = 2
expF = butler.get('raw', detector=detector, exposure=expIdF, instrument=instrument)
biasExpF = butler.get('bias', detector=detector, exposure=expIdF, instrument=instrument)
expD = butler.get('raw', detector=detector, exposure=expIdD, instrument=instrument)
biasExpD = butler.get('bias', detector=detector, exposure=expIdD, instrument=instrument)
ptc = butler.get('ptc', detector=detector, instrument=instrument,
                 collections="u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a")
isrResultF = isrTask.run(expF, bias=biasExpF, ptc=ptc)
arrF = isrResultF.exposure.image.array
isrResultD = isrTask.run(expD, bias=biasExpD, ptc=ptc)
arrD = isrResultD.exposure.image.array

In [ ]:
arrF.shape

In [ ]:
for i in range(arrF.shape[0]):
    for j in range(arrF.shape[1]):
        if arrF[i,j] > 30000 and arrD[i,j] > 500:
            print(i,j, arrD[i,j])


In [ ]:
instrument = "LSSTComCam"
detector = 2
centers = [(315, 289), (3159, 558), (2533, 1442), (1438, 2159)]
expIds = [2024072900232, 2024072900244, 2024072900239, 2024072900261, 2024072900237,
         2024072900187, 2024072900275, 2024072900347, 2024072900242, 2024072900247,
         2024072900212, 2024072900252]
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butler.get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        expTimes[n,m] = mData['EXPTIME']
        exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
        biasExp = butler.get('bias', detector=detector, exposure=expId, instrument=instrument)
        ptc = butler.get('ptc', detector=detector, instrument=instrument,
                         collections="u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a")
        isrResult = isrTask.run(exp, bias=biasExp, ptc=ptc) # This runs the ISR
        peakValues[n,m] = isrResult.exposure.image.array[center[1], center[0]]
        fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
        fig.suptitle(f"Warm pixels, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center}", fontsize=18)
        plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Warm_Pixel_{center[0]}_{center[1]}_{expId}.png")


In [ ]:
%matplotlib inline
colors = ['red', 'blue', 'green', 'orange']
fig = plt.figure(figsize=(8,5))
for i in range(len(centers)):
    plt.plot(expTimes[i,:], peakValues[i,:], marker='x', label=f"{centers[i]}", color=colors[i])
    plt.scatter([30.0], [arrD[centers[i][1], centers[i][0]]], marker='o', color=colors[i], label='')
plt.scatter([30.0], [arrD[centers[0][1], centers[0][0]]], marker='o', color = 'red', label='Dark')
plt.legend()
plt.xlabel("Exposure time (seconds)")
plt.ylabel("Peak pixel flux (e-)")
plt.title(f"Warm Pixel Peak Flux, ComCam Detector {detector}")
plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Warm_Pixel_Flux_vs_ExpTime_04Sep24.png")

## Check images with different filters

In [ ]:
instrument = "LSSTComCam"
detector = 2
centers = [(3159, 558)]
expIds = [2024080800120, 2024080800055, 2024080800025, 2024080800095, 2024080800145, 2024080800180]
filters = ['U', 'G', 'R', 'I', 'Z', 'Y']
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butler.get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        expTimes[n,m] = mData['EXPTIME']
        exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
        biasExp = butler.get('bias', detector=detector, exposure=expId, instrument=instrument)
        ptc = butler.get('ptc', detector=detector, instrument=instrument,
                         collections="u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a")
        isrResult = isrTask.run(exp, bias=biasExp, ptc=ptc) # This runs the ISR
        peakValues[n,m] = isrResult.exposure.image.array[center[1], center[0]]
        fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
        fig.suptitle(f"Warm pixels, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center} Filter={filters[m]}", fontsize=18)
        plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Warm_Pixel_{center[0]}_{center[1]}_{expId}_{filters[m]}.png")


In [ ]:
instrument = "LSSTComCam"
detector = 2
centers = [(3159, 558)]
expIds = [2024080800120, 2024080800055, 2024080800025, 2024080800095, 2024080800145, 2024080800180]
filters = ['U', 'G', 'R', 'I', 'Z', 'Y']
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butler.get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        expTimes[n,m] = mData['EXPTIME']
        exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
        biasExp = butler.get('bias', detector=detector, exposure=expId, instrument=instrument)
        ptc = butler.get('ptc', detector=detector, instrument=instrument,
                         collections="u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a")
        isrResult = isrTask.run(exp, bias=biasExp, ptc=ptc) # This runs the ISR
        peakValues[n,m] = isrResult.exposure.image.array[center[1], center[0]]
        fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
        fig.suptitle(f"Warm pixels, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center} Filter={filters[m]}", fontsize=18)
        plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Warm_Pixel_{center[0]}_{center[1]}_{expId}_{filters[m]}.png")


In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
instrument = "LSSTComCam"
detector = 2
centers = [(3159, 558)]
expIds = [2024072900232, 2024072900244, 2024072900239, 2024072900261, 2024072900237,
         2024072900187, 2024072900275, 2024072900347, 2024072900242, 2024072900247,
         2024072900212, 2024072900252]
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butler.get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        expTimes[n,m] = mData['EXPTIME']
        exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
        isrResult = isrTask.run(exp) # This runs the ISR
        print(expId, mData['EXPTIME'], np.nanmedian(isrResult.exposure.image.array))
        #peakValues[n,m] = isrResult.exposure.image.array[center[1], center[0]]
        #fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
        #fig.suptitle(f"Warm pixels, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center}", fontsize=18)
        #plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Warm_Pixel_{center[0]}_{center[1]}_{expId}.png")


In [ ]:
butler0 = Butler('/repo/main', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butler1 = Butler('/repo/embargo_old', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butlers = [butler0, butler1]
instrument = "LSSTComCam"
detector = 2
centers = [(3159, 558)]
expIds = [2021040100045, 2024072900275]
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butlers[m].get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        expTimes[n,m] = mData['EXPTIME']
        exp = butlers[m].get('raw', detector=detector, exposure=expId, instrument=instrument)
        isrResult = isrTask.run(exp) # This runs the ISR
        peakValues[n,m] = isrResult.exposure.image.array[center[1], center[0]]
        fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
        fig.suptitle(f"Vampire pixel, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center}", fontsize=18)
        plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Vampires_{center[0]}_{center[1]}_{expId}.png")


In [ ]:
instrument = "LSSTComCam"
detector = 2
centers = [(3159, 558)]
expIds = [2022051900090, 2022053100041]
Vbb = ['Vbb_OFF', 'Vbb_ON']
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butlers[0].get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        exp = butlers[0].get('raw', detector=detector, exposure=expId, instrument=instrument)
        isrResult = isrTask.run(exp) # This runs the ISR
        fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
        fig.suptitle(f"Vampire pixel, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center}, {Vbb[m]}", fontsize=18)
        plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Vampires_{center[0]}_{center[1]}_{expId}_{Vbb[m]}.png")


In [ ]:
butler0 = Butler('/repo/main', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butler1 = Butler('/repo/embargo_old', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butler2 = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib", "LSSTComCam/nightlyValidation"])
butlers = [butler2, butler1]
instrument = "LSSTComCam"
detector = 2
centers = [(3159, 558)]
expIds = [2024111900013, 2024071100061]
Vbb = ['Vbb_OFF', 'Vbb_ON']
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butlers[m].get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        exp = butlers[m].get('raw', detector=detector, exposure=expId, instrument=instrument)
        isrResult = isrTask.run(exp) # This runs the ISR
        fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
        fig.suptitle(f"Vampire pixel, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center}, {Vbb[m]}", fontsize=18)
        plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Vampires_{center[0]}_{center[1]}_{expId}_{Vbb[m]}.png")


In [ ]:
butler0 = Butler('/repo/main', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butler1 = Butler('/repo/embargo_old', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butler2 = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib", "LSSTComCam/nightlyValidation"])
butlers = [butler0, butler1, butler2]
instrument = "LSSTComCam"
detector = 3
centers = [(1538, 1711), (2005, 1676), (1900, 1869), (3429, 2819)]
expIds = [2024111900027]
Vbb = ['Vbb_OFF', 'Vbb_ON']
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butlers[2].get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        exp = butlers[2].get('postISRCCD', detector=detector, exposure=expId, instrument=instrument)
        fig = localPlot(exp.image.array, center=center, extent=(30,30), stretch='ccs')
        fig.suptitle(f"Vampire pixel, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center}", fontsize=18)
        plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Vampires_{center[0]}_{center[1]}_{expId}.png")


In [ ]:
butler0 = Butler('/repo/main', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butler1 = Butler('/repo/embargo_old', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butler2 = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])
butlers = [butler0, butler1, butler2]
instrument = "LSSTComCam"
detector = 2
centers = [(315, 289), (3159, 558), (2533, 1442), (1438, 2159)]
expIds = [2024112300013, 2024112300014]
expTimes = np.zeros([len(centers), len(expIds)])
peakValues = np.zeros([len(centers), len(expIds)])
for n, center in enumerate(centers):
    for m, expId in enumerate(expIds):
        mData = butlers[2].get('raw.metadata', detector=detector, exposure=expId, instrument=instrument)
        expTimes[n,m] = mData['EXPTIME']
        exp = butlers[2].get('raw', detector=detector, exposure=expId, instrument=instrument)
        isrResult = isrTask.run(exp) # This runs the ISR
        peakValues[n,m] = isrResult.exposure.image.array[center[1], center[0]]
        fig = localPlot(isrResult.exposure.image.array, center=center, extent=(30,30), stretch='ccs')
        fig.suptitle(f"Vampire pixel, {expId}, expTime={mData['EXPTIME']} sec\n Detector = {detector}, Center = {center}", fontsize=18)
        plt.savefig(f"/home/c/cslage/u/ComCam/warm_pixels/Vampires_{center[0]}_{center[1]}_{expId}.png")
